In [1]:
import pandas as pd
import sqlite3
from collections import defaultdict
import json

# read data into DataFrame from db

con = sqlite3.connect("db/olympic_data.db")

sql = f"""
    SELECT * FROM athlete_events
    """

athlete_data = pd.read_sql(sql, con)
athlete_data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


In [2]:
# making an average table for males and females
# uploading it to sqlite DB
# this section is BY TEAM
maleData = athlete_data[athlete_data["Sex"] == "M"]
femaleData = athlete_data[athlete_data["Sex"] == "F"]

yearTeamGroupM = maleData.groupby(["Year", "Team"])
yearTeamGroupF = femaleData.groupby(["Year", "Team"])

groupAvgM = yearTeamGroupM[["Age", "Height", "Weight"]].mean()
groupAvgF = yearTeamGroupF[["Age", "Height", "Weight"]].mean()

groupAvgM = groupAvgM.reset_index()
groupAvgF = groupAvgF.reset_index()

athlete_avg = groupAvgM.merge(groupAvgF, on=["Year", "Team"],
                             how="outer", suffixes=("_M", "_F"))

con = sqlite3.connect("db/olympic_data.db")
athlete_avg.to_sql("athlete_avg_team", con, if_exists="replace", index=False)

con.commit()

# this section is BY NOC

yearTeamGroupM = maleData.groupby(["Year", "NOC"])
yearTeamGroupF = femaleData.groupby(["Year", "NOC"])

groupAvgM = yearTeamGroupM[["Age", "Height", "Weight"]].mean()
groupAvgF = yearTeamGroupF[["Age", "Height", "Weight"]].mean()

groupAvgM = groupAvgM.reset_index()
groupAvgF = groupAvgF.reset_index()

athlete_avg = groupAvgM.merge(groupAvgF, on=["Year", "NOC"],
                             how="outer", suffixes=("_M", "_F"))

con = sqlite3.connect("db/olympic_data.db")
athlete_avg.to_sql("athlete_avg_noc", con, if_exists="replace", index=False)

con.commit()
con.close()

In [6]:
# creating Male and female age dataframes
# uploading as seperate tables to the database
# this is to be used for the box plots

yearAge = athlete_data[["Name","Sex", "Age", "Year"]]
yearAge = yearAge.drop_duplicates(subset=["Name", "Year"])

yearAge = yearAge[["Sex", "Age", "Year"]]

yearAge = yearAge.dropna(subset=["Age"])

yearAgeM = yearAge[yearAge["Sex"] == "M"]
yearAgeF = yearAge[yearAge["Sex"] == "F"]

yearAgeM = yearAgeM.sort_values(by="Year").reset_index(drop=True)
yearAgeF = yearAgeF.sort_values(by="Year").reset_index(drop=True)

con = sqlite3.connect("db/olympic_data.db")

yearAgeM.to_sql("yearAgeM", con, if_exists="replace", index=False)
yearAgeF.to_sql("yearAgeF", con, if_exists="replace", index=False)

con.commit()
con.close()

In [2]:
# query male dataframe from db
#jsonify items to be returned to front end for box plot

con = sqlite3.connect("db/olympic_data.db")

sql = f"""
    SELECT * FROM yearAgeM
    """
yearAvgM = pd.read_sql(sql, con)


query_dict = defaultdict(list)

for index, row in yearAvgM.iterrows():
    query_dict[row["Year"]].append(row["Age"])
    
clean_dict = dict(query_dict)

clean_list = [{k:v} for k, v in clean_dict.items()]


In [7]:
con = sqlite3.connect("db/olympic_data.db")

sql = f"""
    SELECT * FROM yearAgeM
    """
yearAvgM = pd.read_sql(sql, con)
yearAvgM

,Sex,Age,Year
0,M,20.0,1896
1,M,21.0,1896
2,M,25.0,1896
3,M,21.0,1896
4,M,23.0,1896
5,M,21.0,1896
6,M,22.0,1896
7,M,23.0,1896
8,M,25.0,1896
9,M,26.0,1896
